# 01 Import libraries

In [ ]:
import numpy as np
import torch


device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using device: {device}")

Using device: cuda


In [ ]:
import pandas as pd

In [ ]:
import importlib

## Import our own modules

In [ ]:
!rm -r Learning-the-Optimal-Solution-Path
!git clone https://github.com/Cumberkid/Learning-the-Optimal-Solution-Path.git

rm: cannot remove 'Learning-the-Optimal-Solution-Path': No such file or directory
Cloning into 'Learning-the-Optimal-Solution-Path'...
remote: Enumerating objects: 1664, done.
remote: Counting objects: 100% (634/634), done.
remote: Compressing objects: 100% (275/275), done.
remote: Total 1664 (delta 481), reused 479 (delta 359), pack-reused 1030
Receiving objects: 100% (1664/1664), 13.23 MiB | 20.72 MiB/s, done.
Resolving deltas: 100% (1118/1118), done.


(Using Colab)

In [ ]:
import sys

In [ ]:
# Add the parent directory to sys.path
sys.path.append('/content/Learning-the-Optimal-Solution-Path')

In [ ]:
import lib
importlib.reload(lib)

<module 'lib' from '/content/Learning-the-Optimal-Solution-Path/lib/__init__.py'>

In [ ]:
from lib.fast_tensor_data_loader import FastTensorDataLoader
from lib.ngs.naive_grid_search import naive_grid_search
from lib.ngs.utils_ngs import get_losses
from lib.ngs.loss_fn import reg_exp_weighted_logit

# 02 Load data

In [ ]:
# file path for Colab. May need to change this
X_df = pd.read_csv('/content/Learning-the-Optimal-Solution-Path/experiments/fair-regression/data/X_processed.csv')
y_df = pd.read_csv('/content/Learning-the-Optimal-Solution-Path/experiments/fair-regression/data/y_processed.csv')

In [ ]:
X = np.array(X_df)
y = np.array(y_df).squeeze()

In [ ]:
train_X = torch.tensor(X, dtype=torch.float32)
train_y = torch.tensor(y, dtype=torch.float32)

In [ ]:
# full gradient descent uses all data points
GD_data_loader = FastTensorDataLoader(train_X, train_y, batch_size=1000, shuffle=True, )
# stochastic gradient descent uses mini-batch
SGD_data_loader = FastTensorDataLoader(train_X, train_y, batch_size=20, shuffle=True, )
# test data
test_data_loader = FastTensorDataLoader(train_X, train_y, batch_size=1000, shuffle=False, )

# 03 Compute the true solution path

We use the Naive Grid Search with $2^{10}$ grids, trained by full gradient descent with tuned learning rate = $2$ and 5000 iterations on each grid, to generate a true solution path.

In [ ]:
lam_max = 100
lam_min = 0
input_dim = X.shape[1]
loss_fn = reg_exp_weighted_logit

In [ ]:
num_grid = 2**10
lambdas = np.linspace(lam_max, lam_min, num_grid)
fine_delta_lam = (lam_max - lam_min)/(num_grid - 1)

In [ ]:
epochs = 5000
lr = 1 # previously tuned

In [ ]:
total_itr, reg_params, intercepts, weights, grid_pass_error = naive_grid_search(lam_min, lam_max, num_grid,
                                epochs, loss_fn, GD_data_loader, input_dim, lr=lr, device=device)

In [ ]:
losses = get_losses(lam_min, lam_max, fine_delta_lam, intercepts,
                              weights, reg_params, test_data_loader, loss_fn, device)

In [ ]:
thetas = np.array(weights)
print(thetas.shape)

(1024, 45)


In [ ]:
headers = ['losses', 'theta_0', 'theta_1', 'theta_2', 'theta_3',
            'theta_4', 'theta_5', 'theta_6', 'theta_7', 'theta_8',
            'theta_9', 'theta_10', 'theta_11', 'theta_12', 'theta_13',
            'theta_14', 'theta_15', 'theta_16', 'theta_17', 'theta_18',
            'theta_19', 'theta_20', 'theta_21', 'theta_22', 'theta_23',
            'theta_24', 'theta_25', 'theta_26', 'theta_27', 'theta_28',
            'theta_29', 'theta_30', 'theta_31', 'theta_32', 'theta_33',
            'theta_34', 'theta_35', 'theta_36', 'theta_37', 'theta_38',
            'theta_39', 'theta_40', 'theta_41', 'theta_42', 'theta_43',
            'theta_44', 'theta_45']

exact_soln_list = pd.DataFrame(np.column_stack((losses, intercepts, thetas)), columns=headers)

# Save the DataFrame to a CSV file
exact_soln_list.to_csv('exact_soln_list.csv', index=False)

In [ ]:
# Read the CSV file into a DataFrame
truth = pd.read_csv('exact_soln_list.csv')

# Display the DataFrame
truth

In [ ]:
# Read the CSV file into a DataFrame
truth = pd.read_csv('exact_soln_list.csv')

# Display the DataFrame
truth

In [ ]:
selected_columns = ['theta_0', 'theta_1', 'theta_2', 'theta_3', 'theta_4',
                    'theta_5', 'theta_6', 'theta_7', 'theta_8', 'theta_9',
                    'theta_10', 'theta_11', 'theta_12', 'theta_13', 'theta_14',
                    'theta_15', 'theta_16', 'theta_17', 'theta_18', 'theta_19',
                    'theta_20', 'theta_21', 'theta_22', 'theta_23', 'theta_24',
                    'theta_25', 'theta_26', 'theta_27', 'theta_28', 'theta_29',
                    'theta_30', 'theta_31', 'theta_32', 'theta_33', 'theta_34',
                    'theta_35', 'theta_36', 'theta_37', 'theta_38', 'theta_39',
                    'theta_40', 'theta_41', 'theta_42', 'theta_43', 'theta_44',
                    'theta_45']
true_thetas = truth[selected_columns].to_numpy()
true_losses = truth['losses'].to_numpy()